In [1]:
!pip install -q https://github.com/PatrickKudo/flappy-bird-gymnasium/archive/refs/heads/main.zip
!pip install -q optuna

     / 41.4 MB 17.0 MB/s 0:00:03
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 12.1 MB/s eta 0:00:00


In [1]:
%%writefile run_trial.py
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

import math
import random
import time
from collections import namedtuple, deque
from itertools import count
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import pygame
import gymnasium
import flappy_bird_gymnasium
import torch
import torch.nn as nn
import torch.optim as optim
import gymnasium as gym
import argparse
from statistics import mean, stdev


class DuelingDQN(nn.Module):
    def __init__(self, input_channels, input_length, action_space, model_size):
        super(DuelingDQN, self).__init__()

        if model_size == "small":
          cnn_hidden = 8
          dense_hidden = 64
          stream_hidden = 64
        elif model_size == "medium":
          cnn_hidden = 16
          dense_hidden = 128
          stream_hidden = 128
        elif model_size == "large":
          cnn_hidden = 32
          dense_hidden = 256
          stream_hidden = 256
        else:
          raise ValueError("Invalid model size")

        # 1D Convolutional feature extractor
        self.conv1d_features = nn.Sequential(
            nn.Conv1d(input_channels, cnn_hidden, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Conv1d(cnn_hidden, 16, kernel_size=2, stride=1),
            nn.ReLU(),
            nn.Flatten()
        )

        # Fully Connected feature extractor
        self.dense_features = nn.Sequential(
            nn.Linear(input_length * input_channels, dense_hidden),
            nn.ReLU(),
            nn.Dropout(0.1)
        )

        # Combined feature size
        combined_feature_size = 144 + dense_hidden

        # Value stream
        self.value_stream = nn.Sequential(
            nn.Linear(combined_feature_size, stream_hidden),
            nn.ReLU(),
            nn.Linear(stream_hidden, 1)
        )

        # Advantage stream
        self.advantage_stream = nn.Sequential(
            nn.Linear(combined_feature_size, stream_hidden),
            nn.ReLU(),
            nn.Linear(stream_hidden, action_space)
        )

    def forward(self, state):
        conv_features = self.conv1d_features(state.unsqueeze(1))
        dense_features = self.dense_features(state.view(state.size(0), -1))
        combined_features = torch.cat((conv_features, dense_features), dim=1)

        value = self.value_stream(combined_features)
        advantages = self.advantage_stream(combined_features)
        # from the paper: Q(s, a; θ, α, β) = V (s; θ, β) + (A(s, a; θ, α) − mean(A(s, a'; θ, α))).
        qvals = value + (advantages - advantages.mean(dim=1, keepdim=True))
        return qvals


# Replay memory
class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)



# Initialize misc. game settings
os.environ["SDL_VIDEODRIVER"] = "dummy"
os.environ["SDL_AUDIODRIVER"] = "dummy"
pygame.init()

# Establish Flappy Bird environment
env = gymnasium.make(
    "FlappyBird-v0", audio_on=False, render_mode="rgb_array", use_lidar=False
)

# set up matplotlib display functionality
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# Check if GPU is available to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("cuda is available: ", torch.cuda.is_available())

# Single transition of environment: map state-action pairs to rewards
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))



# Initialize the parser
parser = argparse.ArgumentParser()

# Adding arguments
parser.add_argument("--batch_size", type=int, help="The batch size for the model training", required=True)
parser.add_argument("--lr", type=float, help="Learning rate", required=True)
parser.add_argument("--gamma", type=float, help="Gamma value used in optimizations", required=True)
parser.add_argument("--tau", type=float, help="Target network update rate", required=True)
parser.add_argument("--model_size", type=str, help="Model size", required=True)

# Parse the arguments
args = parser.parse_args()

BATCH_SIZE = args.batch_size
GAMMA = args.gamma
LR = args.lr
TAU = args.tau
MODEL_SIZE = args.model_size

EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000

print("Batch size:", BATCH_SIZE)
print("Learning rate:", LR)
print("Gamma:", GAMMA)
print("Tau:", TAU)

# Get number of actions from gym action space
n_actions = env.action_space.n
# Get the number of state observations
state, info = env.reset()
n_observations = len(state)

input_channels = 1
input_length = 12
policy_net = DuelingDQN(input_channels, input_length, n_actions, MODEL_SIZE).to(device)
target_net = DuelingDQN(input_channels, input_length, n_actions, MODEL_SIZE).to(device)
target_net.load_state_dict(policy_net.state_dict())

# Define optimizer
optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
# Set replay limit
memory = ReplayMemory(2000)

# Initialize step counter
steps_done = 0

def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            return policy_net(state).max(1).indices.view(1, 1)
    else:
        return torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)

episode_durations = []

def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    batch = Transition(*zip(*transitions))
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)
    state_action_values = policy_net(state_batch).gather(1, action_batch)
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()


# Determine episodes based on GPU availability
if torch.cuda.is_available():
    num_episodes = 2000
else:
    num_episodes = 50

# Start training loop
for i_episode in range(num_episodes):
    # Initialize the environment and get its state
    state, info = env.reset()
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    for t in count():
        action = select_action(state)
        observation, reward, terminated, truncated, _ = env.step(action.item())
        reward = torch.tensor([reward], device=device)
        done = terminated or truncated

        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)

        if done:
            episode_durations.append(t + 1)
            break

print("std: ", stdev(episode_durations))
print("mean: ", mean(episode_durations))
print("maxL: ", max(episode_durations))

with open('dqcnn_results.txt', 'w') as f:
    f.write(f"{mean(episode_durations)}")

Overwriting run_trial.py


In [2]:
import optuna
import gc
from statistics import mean, stdev


def objective(trial):
    # Define Hyperparameters
    lr = trial.suggest_float('lr', 1e-5, 1e-3, log=True)
    BS = trial.suggest_categorical('batch_size',
                                            [64, 128, 256])
    G = trial.suggest_float('gamma', 0.88, 0.99)
    T = trial.suggest_float('tau', 0.001, 0.01)

    # Run python script with parameters
    !python run_trial.py --lr $lr --batch_size $BS --gamma $G --tau $T

    # Clean up memory
    gc.collect()

    with open('dqcnn_results.txt', 'r') as f:
        mean_reward = float(f.read())
    print(f"Mean reward: {mean_reward}")
    return mean_reward


In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=32)

print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# save tuna results
study.trials_dataframe().to_csv('dqcnn_tuna_results.csv')
!cp dqcnn_tuna_results.csv "/content/drive/MyDrive/MSAI/spr24/RL/"
# visualize
optuna.visualization.plot_optimization_history(study)

[I 2024-04-24 12:54:39,446] A new study created in memory with name: no-name-057fb724-8c23-4a37-aa58-d5583220ba1c


pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
cuda is available:  True
Batch size: 128
Learning rate: 0.00017564655025175579
Gamma: 0.9681211331147841
Tau: 0.008001256929323113
std:  40.001301351392335
mean:  78.6815
maxL:  355


[I 2024-04-24 13:13:14,306] Trial 0 finished with value: 78.6815 and parameters: {'lr': 0.00017564655025175579, 'batch_size': 128, 'model_size': 'medium', 'gamma': 0.9681211331147841, 'tau': 0.008001256929323113}. Best is trial 0 with value: 78.6815.


Mean reward: 78.6815
pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
cuda is available:  True
Batch size: 256
Learning rate: 0.0009121957338150469
Gamma: 0.9313249608727731
Tau: 0.009498521384071975
std:  41.33713316485729
mean:  81.849
maxL:  348


[I 2024-04-24 13:36:23,217] Trial 1 finished with value: 81.849 and parameters: {'lr': 0.0009121957338150469, 'batch_size': 256, 'model_size': 'large', 'gamma': 0.9313249608727731, 'tau': 0.009498521384071975}. Best is trial 1 with value: 81.849.


...

Mean reward: 79.5695
pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
cuda is available:  True
Batch size: 64
Learning rate: 0.0001765912168805269
Gamma: 0.9667164701393973
Tau: 0.008139815162126472
std:  30.991005343584824
mean:  70.447
maxL:  348


[I 2024-04-24 23:13:56,418] Trial 29 finished with value: 70.447 and parameters: {'lr': 0.0001765912168805269, 'batch_size': 64, 'model_size': 'medium', 'gamma': 0.9667164701393973, 'tau': 0.008139815162126472}. Best is trial 2 with value: 86.455.


Mean reward: 70.447
pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
cuda is available:  True
Batch size: 128
Learning rate: 0.00011986859983406246
Gamma: 0.9544037142461727
Tau: 0.007397846635917882
std:  36.9021993539488
mean:  75.413
maxL:  464


[I 2024-04-24 23:31:47,788] Trial 30 finished with value: 75.413 and parameters: {'lr': 0.00011986859983406246, 'batch_size': 128, 'model_size': 'medium', 'gamma': 0.9544037142461727, 'tau': 0.007397846635917882}. Best is trial 2 with value: 86.455.


Mean reward: 75.413
pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
cuda is available:  True
Batch size: 256
Learning rate: 0.00011912364845963011
Gamma: 0.9419522427480308
Tau: 0.009235307280301074
